# 传统数据库

上篇文章：聊聊数据库~开篇 <https://www.cnblogs.com/dotnetcrazy/p/9690466.html>

本来准备直接开讲NoSQL的（当时开篇就是说的NoSQL）考虑到有些同志可能连MySQL系都没接触过，所以我们2019说数据系的时候预计从`MySQL`（穿插`MSSQL`）开始，这篇文章就当试水篇，效果好就继续往下写~（这篇偏理论和运维）

## 1.1.MariaDB and MySQL

官方文档：`https://mariadb.com/kb/zh-cn/mariadb`

目前主流：`MySQL 5.7` or **`MariaDB 5.5.x`**（推荐）

多一句嘴，`MySQL`当年被`Oracle`收购后，`MySQL之父`觉得靠`Oracle`维护`MySQL`很不靠谱，然后就跳槽弄了个`MariaDB`（很多`Oracle`竞争对手扶持着），目前`MariaDB`是发展最快的`MySQL`分支版本（PS：`MySQL`现在是双协议了，大部分公司用的版本都是`<=5.7`）

然后得说下迁移问题：`MySQL 5.x`到 `MariaDB 5.x`基本上是无缝的，`MariaDB最新稳定版为：MariaDB 5.5`

`MariaDB`与`MySQL`兼容性可以查看：
https://mariadb.com/kb/en/library/mariadb-vs-mysql-compatibility

PS：国内比较火的还有阿里的`MySQL分支`：`https://github.com/alibaba/AliSQL`

不谈其他的，咱们看看它们开发的积极程度就知道为什么`MariaDB`是主流了
![1.积极.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181226114927826-720399690.png)

## 1.2.MariaDB部署

### 1.环境配置和初始化

安装很简单，以`CentOS`为例：
![1.CentOS.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181226231314430-1618955434.png)

```shell
systemctl start mariadb.service   # 启动MariaDB
systemctl enable mariadb.service  # 设置开机启动

systemctl stop mariadb.service    # 停止MariaDB
systemctl restart mariadb.service # 重启MariaDB
```

![1.运行.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181227161744210-2130831622.png)

PS：Win安装注意这一步：
![1.win.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181226231342520-1201124099.png)

执行文件简单说明：有时候我们 `ps aux | grep mysql` 的时候，发现运行的并不是`/usr/bin/`下的`mysqld`而是`mysqld_safe`，那这个`mysqld_safe`是啥呢？==> **线程安全的实例**

`MariaDB`的程序组成：`ls /usr/bin | grep mysql`
1. Client：
   - **`mysql`**        命令行客户端
   - **`mysqldump`**    数据库备份用
   - **`mysqladmin`**   远程管理工具
   - **`mysqlbinlog`**  二进制日志管理工具
   - ...
2. Server：
   - **`mysqld_safe`**  线程安全的实例
   - `mysqld_multi` 多实例
   - `mysqld`
   - **`mysql_secure_installation`** 安全初始化工具(记得先启动数据库哦)
   - ...

**`mysql`的账号由两部分组成：`username`@`host`，MySQL客户端连接参数：**
- `-u用户名`：默认为`root`
- `-h服务器主机`：默认为`localhost`
    - `host`用于限制用户可以通过哪些主机连接
    - 支持通配符：
        - `%`匹配任意长度的任意字符：172.16.0.0/16 ==> 172.16.%.%
        - `_`匹配任意单个字符
- `-p密码`：默认为`空`
    - 安装完成后运行`mysql_secure_installation`来设置密码并初始化

很多人安装完成后是这样设置密码的：(**不推荐**)
![1.不推荐.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181226230835330-601019989.png)

**正确打开方式：`mysql_secure_installation`**
![1.安全初始化1.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181226225313373-1042152945.png)

如果允许root远程登录：`Disallow root login remotely? [Y/n] n`
![1.安全初始化2.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181226225701877-196565629.png)

安全初始化后登录图示：
![1.安全初始化后登录.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181226230645032-776720431.png)
![1.基本信息.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181226232704832-766679019.png)

### 2.配置文件

以`MariaDB 5.5.60`为例：
1. Linux：配置文件查找顺序(找不到就往下继续)
   - `/etc/my.cnf` --> **`/etc/mysql/conf.d/*.cnf`** --> `~/.my.cnf`
2. Windows：`MariaDB安装目录/data/my.ini`

PS：一般配置文件都会设置这3个
```shell
# 指定数据库文件存放路径
datadir = /home/dnt/data
# 独立表空间: 每一个表都有一个.frm表描述文件，还有一个.ibd文件
innodb_file_per_table=on
# 不对连接进行DNS解析(省时)
skip_name_resolve=on
```
其他配置`MariaDB`提供了样本：
```shell
[dnt@localhost ~] ls /usr/share/mysql/ | grep .cnf
my-huge.cnf            # 超大内存配置参考
my-innodb-heavy-4G.cnf # 4G内存配置参考
my-large.cnf           # 大内存配置
my-medium.cnf          # 中等内存配置
my-small.cnf           # 小内存配置
```
PS：`thread_concurrency`=`CPU数*2`最佳，**修改配置后记得重启数据库**

### 3.远程访问

1.之前安全初始化的时候把`root`禁止远程登录了，现在我们创建一个其他用户
![1.新增用户.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181227153211084-1542804218.png)

2.给用户权限
![1.给权限.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181227154329334-308689111.png)

3.防火墙放行指定端口
![1.防火墙.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181227154746690-1949205505.png)

4.远程客户端测试一下
![1.成功.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181227132048960-1914298109.png)

Code如下：
```shell
# root账户登录
mysql -uroot -p

# 新增用户
insert into mysql.user(user,host,password) values("用户名","%",password("密码"));

# 刷新设置
flush privileges;

# 分配权限
grant all privileges on 数据库.* to 用户名@"%" identified by "密码";

# 刷新设置
flush privileges;

# 显示服务状态
systemctl status firewalld

# 添加 --permanent永久生效(没有此参数重启后失效)
firewall-cmd --zone=public --add-port=80/tcp --permanent

# 重新载入
firewall-cmd --reload

# 查看
firewall-cmd --zone= public --query-port=80/tcp
# 删除
firewall-cmd --zone= public --remove-port=80/tcp --permanent
```

**SQLServer远程连接**：<https://www.cnblogs.com/dunitian/p/5474501.html>

## MySQL军规（58）

文章结尾贴一节`58`的`MySQL`军规：（**适用于并发量大，数据量大的典型互联网业务**）

### 1.基础规范

1. 表存储引擎必须使用`InnoDB`
2. 表字符集默认使用`utf8`，必要时候使用`utf8mb4`
    - `utf8`通用，无乱码风险，汉字3字节，英文1字节
    - `utf8mb4`是`utf8的超集`，存储4字节时使用（eg：表情符号）
3. **禁止使用存储过程，视图，触发器，Event**
    - 调试，排错，迁移都比较困难，扩展性较差
    - 对数据库性能影响较大，互联网业务，能让站点层和服务层干的事情，不要交到数据库层
4. 禁止在数据库中存储大文件（eg：照片）
    - 可以将大文件存储在对象存储系统，数据库中存储路径
5. 禁止在线上环境做数据库压力测试
    - 测试，开发，线上数据库环境必须隔离
 
### 2.命名规范

1. **库名，表名，列名必须用小写，采用下划线分隔**
    - abc，Abc，ABC都是给自己埋坑
2. 库名，表名，列名必须见名知义，长度不要超过32字符
    - tmp，wushan谁TM知道这些库是干嘛的
3. 库备份必须以bak为前缀，以日期为后缀
    - 从库必须以-s为后缀
    - 备库必须以-ss为后缀
 
### 3.表设计规范

1. 单实例表个数必须控制在`2000`个以内
2. 单表分表个数必须控制在`1024`个以内
3. **表必须有主键，推荐使用`unsigned`整数为主键**
    - 潜在坑：删除无主键的表，如果是row模式的主从架构，从库会挂住
4. 禁止使用外键，如果要保证完整性，应由应用程式实现
    - 外键使得表之间相互耦合，影响`update/delete`等SQL性能
    - 有可能造成死锁，高并发情况下容易成为数据库瓶颈
5. 建议将大字段，访问频度低的字段拆分到单独的表中存储，分离冷热数据
    - 垂直拆分的依据，尽量把长度较短，访问频率较高的属性放在主表里
    - 流量大数据量大时，数据访问要有`service`层，并且`service`层不要通过`join`来获取主表和扩展表的属性
    - 具体可以参考沈剑大牛写的<a href="https://mp.weixin.qq.com/s/ezD0CWHAr0RteC9yrwqyZA" target="_blank">《如何实施数据库垂直拆分》</a>
 
### 4.列设计规范

1. 根据业务区分使用`tinyint`/`int`/`bigint`，分别会占用`1`/`4`/`8`字节
2. 根据业务区分使用`char`/`varchar`（PS：没有MSSQL里的`nvarchar`）
    - 字段长度固定，或者长度近似的业务场景，适合使用`char`，**能够减少碎片，查询性能高**
    - 字段长度相差较大，或者更新较少的业务场景，适合使用`varchar`，能够**减少空间**
3. 根据业务区分使用`datetime`/`timestamp`
    - `datetime`占用5个字节，`timestamp`占用4个字节
    - 存储年使用`year`，存储日期使用`date`，存储时间使用`datetime`
4. **必须把字段定义为`NOT NULL`并设默认值**
    - NULL需要更多的存储空间
    - NULL的列使用索引，索引统计，值都更加复杂，MySQL更难优化
    - NULL只能采用IS NULL或者IS NOT NULL，而在=/!=/in/not in时有大坑
5. **使用`int unsigned`存储`IPv4`**，不要用`char(15)`
6. **使用`varchar(20)`存储手机号，不要使用整数**
    - 手机号不会用来做数学运算
    - `varchar`可以模糊查询(eg：like ‘138%’)
    - 牵扯到国家代号，可能出现`+、-、()`等字符，eg：`+86`
7. 使用`tinyint`来代替`enum`
    - `enum`增加新值要进行`DDL`操作

### 5.索引规范（常用）

1. 唯一索引使用`uniq_字段名`来命名
2. 非唯一索引使用`idx_字段名`来命名
3. **单张表索引数量建议控制在5个以内**
    - 互联网高并发业务，太多索引会影响写性能
    - 异常复杂的查询需求，可以选择`ES`等更为适合的方式存储
    - `生成执行计划时，如果索引太多，会降低性能，并可能导致MySQL选择不到最优索引`
4. **组合索引字段数不建议超过5个**
    - 如果5个字段还不能极大缩小row范围，八成是设计有问题
5. **不建议在频繁更新的字段上建立索引**
6. **非必要不要进行`join`查询，如果要进行`join`查询，被`join`的字段必须类型相同，并建立索引**
    - `join`字段类型不一致容易导致全表扫描
7. 理解组合索引最左前缀原则，避免重复建设索引
    - 如果建立了`(a,b,c)`，相当于建立了`(a)`, `(a,b)`, `(a,b,c)`
 
### 6.SQL规范（常用）

1. **禁止使用`select *`，只获取必要字段**
    - 指定字段能有效利用索引覆盖
    - `select *`会增加`cpu/io/内存/带宽`的消耗
    - `指定字段查询，在表结构变更时，能保证对应用程序无影响`
2. **`insert`必须指定字段，禁止使用`insert into T values()`**
    - 指定字段插入，在表结构变更时，能保证对应用程序无影响
3. **隐式类型转换会使索引失效，导致全表扫描**（很重要）
4. 禁止在`where`条件列使用函数或者表达式
    - 导致不能命中索引，全表扫描
5. 禁止负向查询以及`%`开头的模糊查询
    - 导致不能命中索引，全表扫描
6. 禁止大表`join`和`子查询`
7. **同一个字段上的`or`必须改写为`in`，`in`的值必须少于50个**
8. 应用程序必须捕获SQL异常（方便定位线上问题）

课后思考：为什么`select uid from user where phone=13811223344`不能命中phone索引？

课后拓展：
```
MyISAM与InnoDB两者之间区别与选择
https://www.cnblogs.com/y-rong/p/5309392.html
https://www.cnblogs.com/y-rong/p/8110596.html

了解下Mysql的间隙锁及产生的原因
https://www.cnblogs.com/wt645631686/p/8324671.html

grant授权和revoke回收权限
https://www.cnblogs.com/kevingrace/p/5719536.html

centos7自带数据库MariaDB重启和修改密码
https://blog.csdn.net/shachao888/article/details/50341857

MySQL添加用户、删除用户与授权
https://www.cnblogs.com/wanghetao/p/3806888.html

深度认识 Sharding-JDBC：做最轻量级的数据库中间层
https://my.oschina.net/editorial-story/blog/888650
```

## 1.3.MySQL部署

之前有园友说，为啥不顺便说说`UbuntuServer`的部署呢？呃。。。一般来说公司服务器都是`CentOS`的占大多数，然后`UbuntuServer`更多的是个人云服务比较多（**推荐初创公司使用**），毕竟它们两个系统追求的不太一样，一个是追求稳（部署麻烦），一个是追求软件尽量新的情况下稳定（更新太快）

那么长话短说，步入正轨：

### 1.Ubuntu最常见的问题

Ubuntu不得不说的就是这个**`apt`出问题的处理** ：（换源就不说了`/etc/apt/sources.list`）
```shell
# 一般删除这几个锁文件，然后再重新配置下就可以了
sudo rm /var/lib/dpkg/lock
sudo rm /var/lib/dpkg/lock-frontend
sudo rm /var/lib/apt/lists/lock
sudo rm /var/cache/apt/archives/lock
# 简写（千万注意空格，不然你就是rm -rf / + 跑路了）
# sudo rm /var/lib/apt/lists/lock /var/cache/apt/archives/lock /var/lib/dpkg/lock sudo rm /var/lib/dpkg/lock-frontend

# 重新配置下
sudo dpkg --configure -a
```
### 2.安装注意

然后说下，为啥推荐Ubuntu使用MySQL：安装MariaDB会出现一些莫名的错误(可能是Ubuntu内部依赖了部分MySQL组件),所以推荐使用`MySQL`，毕竟同是`5.x`用起来基本上差不多，安装过程和之前说的`CentOS 下 MariaDB`差不多，所有命令前加个**`sudo`**

1.安装比较简单：`sudo apt install mysql-server`
![2.1.Ubuntu.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181230101637706-882392574.png)

2.允许远程连接：`注释掉 bind-address=127.0.0.1`
![2.2.远程连接.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181230101658414-1379371907.png)

3.关于为什么是这个路径的说明：`sudo vi /etc/mysql/mysql.conf.d/mysqld.cnf`
![2.3.为什么是这个路径.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181230101715440-931363578.png)

4.所有配置修改都需要重新启动下：`sudo systemctl restart mysql`
![2.4.记得重启一下](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181230101808123-1679185302.png)

5.第一次初始化和MariaDB不太一样：`sudo mysql_secure_installation`（其他一路`y`即可）

需要选下你设置root密码的复杂度：（一般1就可以了，就算设置再复杂，入了系统也是虚的）
![2.5.初始化-注意下密码复杂程度](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181230101832777-1084161641.png)

PS：可以看看拓展文章：<a href="https://www.cnblogs.com/super-zhangkun/p/9435974.html" target="_blank">Ubuntu16安装mysql5.7未提示输入密码，安装后修改mysql默认密码</a> 和 <a href="https://blog.csdn.net/hello_world_qwp/article/details/79551789" target="_blank">【不推荐】修改mysql密码策略</a>

6.然后输入密码你就可以登录了`sudo mysql -uroot -p`（PS：你直接`sudo mysql`也可以直接登录）

这边我就不像上节课一步步演示了，直接授权和创建一步走了`grant all privileges on 数据库.* to "用户名"@"%" identified by "复杂密码";`
![2.6.授权创建一步走.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181230105705748-187812189.png)

7.记得`flush privileges;`刷新一下系统表
![2.7.测试.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181230110429390-1200971137.png)


## 1.4.MariaDB特性

首先是**插件式存储引擎(`表类型`)**（MySQL不支持）PS：其实也就是支持更多的存储引擎（包括自定义的）

### 1.标准存储引擎

先说说`MySQL`标准存储引擎(`表类型`)：
1. **`MyISAM`**：只支持`表级锁`，不支持`事务`
2. **`InnoDB`**：支持`事务`、`间隙锁`、`行锁`等等

`MariaDB`对标准存储引擎进行了改造升级：
1. `MyISAM` ==> `Aria`：支持崩溃后的恢复
2. `InnoDB` ==> **`XtraDB`**

还进行了很多扩展并开发了新的功能（也提供了很多测试工具），比如添加一些`NoSQL`的功能（`MSSQL`也扩展了`NoSQL`）



